# Chapter 09: Creating Command-line Interfaces

## 薛宇辰準備
## 鍾聖倫修改
## 2018-05-30


Ⓞ 下週作業：將 Chapter 05中，calc 的 CLI.py 用 Chapter 09 的 argparse module 改寫，並且加上手機上至少另外5項的其他計算功能

## 1. unitconverter的功能介紹

#### 先import unitconverter的核心模組

In [95]:
from unitconverter.Converter import get_table, convert_units

#### 先來看看定義了基礎物件的UnitTable.py
    unit_tables資料夾內的檔案定義了各種不同單位類型的unit_table物件，所有的這些物件都繼承了UnitTable這個class
    在class UnitTable中有下列這些method：
    * convert: 用於單位轉換
    * get_units: 回傳所有支援的單位名稱

## source  → destination

## source  → to_base_unit → base_unit → from_base_unit → destination

In [5]:
class UnitTable(object):
    """
    A class to manage conversion of units for the same quantity (e.g. length,
    volume, mass)
    """

    def __init__(self):
        self.base_unit = None
        self.to_base_unit = {}
        self.from_base_unit = {}


    def convert(self, source, dest, value):
        """
        Converts a value in one unit to another.

        @param source The source unit
        @param dest The destination unit
        @param value The value to convert
        @return The value in the destination unit
        """

        if not (self.can_convert(source) and self.can_convert(dest)):
            raise ValueError('Cannot convert given units')

        if source != self.base_unit:
            value = self.to_base_unit[source](value)

        if dest != self.base_unit:
            value = self.from_base_unit[dest](value)

        return value


    def get_units(self):
        """
        Returns a list of units that this table can convert.

        @return List of valid units
        """

        convertable_units = set(self.to_base_unit.keys()) & set(self.from_base_unit.keys())
        convertable_units.add(self.base_unit)
        return convertable_units

#### 顯示time的unit_table中可以進行轉換的所有單位

In [6]:
get_table('time').get_units()

{'d', 'h', 'm', 'ms', 'ns', 's', 'us', 'y'}

我們看一下這個 lamda function

In [ ]:
from .. import UnitTable

class time(UnitTable.UnitTable):

    def __init__(self):
        UnitTable.UnitTable.__init__(self)

        # Base unit is second
        self.base_unit = 's'

        # Minute
        self.to_base_unit['m']     = lambda x: x * 60.0
        self.from_base_unit['m']   = lambda x: x / 60.0
        
        #  使用時的語法：     value = self.to_base_unit[source](value)
        #  to_ 與 from_ 為倒數關係

        # Hour
        self.to_base_unit['h']     = lambda x: x * 3600.0
        self.from_base_unit['h']   = lambda x: x / 3600.0

        # Day
        self.to_base_unit['d']     = lambda x: x * 86400.0
        self.from_base_unit['d']   = lambda x: x / 86400.0

        # Year
        self.to_base_unit['y']     = lambda x: x * 3.15569e7
        self.from_base_unit['y']   = lambda x: x / 3.15569e7

        # Millisecond
        self.to_base_unit['ms']    = lambda x: x / 1000.0
        self.from_base_unit['ms']  = lambda x: x * 1000.0

        # Microsecond
        self.to_base_unit['us']    = lambda x: x / 1000000.0
        self.from_base_unit['us']  = lambda x: x * 1000000.0

        # Nanosecond
        self.to_base_unit['ns']    = lambda x: x / 1000000000.0
        self.from_base_unit['ns']  = lambda x: x * 1000000000.0


## 關於 lambda function 的使用

            value = self.to_base_unit[source](value)

In [12]:
get_table('time').base_unit

's'

In [13]:
get_table('time').to_base_unit["d"](1)

86400.0

#### 再来看看unitconverter的核心程式Converter.py
    * get_table: 回傳一個table_name所指定的unit_table物件
    * convert_units: 把用戶指定的數值轉換為以指定單位表示的形式

In [16]:
import importlib


def get_table(table_name):
    """
    Returns an instance of a unit table given the name of the table.

    @param table_name Name of table to retrieve
    @return UnitTable instance
    """

    converter_module = importlib.import_module('unitconverter.unit_tables.%s' % table_name)
    converter_class = getattr(converter_module, table_name)()
    return converter_class


def convert_units(table_name, value, value_unit, targets):
    """
    Converts a given value in a unit to a set of target units.

    @param table_name Name of table units are contained in
    @param value Value to convert
    @param value_unit Unit value is currently in
    @param targets List of units to convert to
    @return List of conversions
    """

    table = get_table(table_name)

    results = list()
    for target in targets:
        result = {'dest_unit': target}
        try:
            result['converted_value'] = table.convert(value_unit, target, value)
        except ValueError:
            continue
        results.append(result)

    return results

#### 將200 wh 的能量分別轉換為 j, hph的形式

In [17]:
get_table('energy').get_units()

{'btu', 'cal', 'ev', 'hph', 'j', 'kcal', 'wh'}

In [18]:
convert_units('energy', 200, 'wh', ['j', 'hph'])

[{'converted_value': 719999.99999424, 'dest_unit': 'j'},
 {'converted_value': 0.2682044216834544, 'dest_unit': 'hph'}]

In [19]:
convert_units('time', 1, 'd', ['h', 'm'])

[{'converted_value': 24.0, 'dest_unit': 'h'},
 {'converted_value': 1440.0, 'dest_unit': 'm'}]

## 關於 command line interface (CLI) 有兩種寫法：
- Chapter 05 用 os.argv[**]
- Chapter 09 用 argparse 的方法：將輸入資料塞入物件，並可提供 -h 等 help 的選項

## 2. 撰寫unitconverter的Command-line Interface程式

#### Command-line Interface(CLI)的設定全部都包含在CLI.py中，CLI指令供分為兩層
    * 第一層用於選擇所需要的unit_table的類型
    * 第二層用於選擇list模式或conversion模式
        - list模式用於顯示選定的unit_table可以轉換的所有單位的名稱
        - conversion模式用於單位的轉換

In [20]:
# import required libraries
import argparse
import inspect
from unitconverter.Converter import get_table, convert_units

### 2.1 借助python內建的setup功能，可以方便地構建CLI環境。setup功能全部位於setup.py
    setup.py定義了以下內容：
    * entry_points: 我們可以在CLI界面中輸入"unitconvert"語句來啟動這個程式
    * packages: 需要包含進來的所有程式
    注意：下面的程式無法直接執行

In [2]:
from setuptools import setup

setup(
    name='unitconverter',
    version='0.1.0',
    entry_points={
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
    },  ## 這一行 entry_points 是新加的！
    description='Command line tool for unit conversion',
    classifiers=[
        'Natural Language :: English',
        'Programming Language :: Python :: 3.5',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: 3.7',
    ],
    author='Dan Nixon',
    packages=['unitconverter', 'unitconverter.unit_tables'],
    include_package_data=True, #  to include any data files in package
    zip_safe=False)

SystemExit: usage: ipykernel_launcher.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: ipykernel_launcher.py --help [cmd1 cmd2 ...]
   or: ipykernel_launcher.py --help-commands
   or: ipykernel_launcher.py cmd --help

error: option -f not recognized

C:\Users\User324\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### 先使用"python setup.py install"，安裝完以後方可以啟用命令行工具

In [21]:
! python setup.py install

running install
running bdist_egg
running egg_info
writing unitconverter.egg-info\PKG-INFO
writing top-level names to unitconverter.egg-info\top_level.txt
writing dependency_links to unitconverter.egg-info\dependency_links.txt
writing entry points to unitconverter.egg-info\entry_points.txt
reading manifest file 'unitconverter.egg-info\SOURCES.txt'
writing manifest file 'unitconverter.egg-info\SOURCES.txt'
installing library code to build\bdist.win32\egg
running install_lib
running build_py
creating build\bdist.win32\egg
creating build\bdist.win32\egg\unitconverter
copying build\lib\unitconverter\CLI.py -> build\bdist.win32\egg\unitconverter
copying build\lib\unitconverter\Converter.py -> build\bdist.win32\egg\unitconverter
copying build\lib\unitconverter\UnitTable.py -> build\bdist.win32\egg\unitconverter
creating build\bdist.win32\egg\unitconverter\unit_tables
copying build\lib\unitconverter\unit_tables\angle.py -> build\bdist.win32\egg\unitconverter\unit_tables
copying build\lib\unit

In [ ]:
##
import argparse
import inspect
from Converter import get_table, convert_units


def run_cli(): # 這是在 setup.py 裏面加進 entry_ponts 的執行程式主體
    parser = argparse.ArgumentParser(description='Tool for converting units')

    parser.add_argument( # 第一個參數
        'table',         # 存入物件的 attribute
        metavar='TABLE',
        action='store',
        type=str,        # 資料型態
        help='Unit table to use in conversion'
    )

    ## 接下來的參數可能是 {list, convert} 它們通稱 subparsers
    
    subparsers = parser.add_subparsers(help='operation to be performed')

    list_table_parser = subparsers.add_parser('list')
    list_table_parser.set_defaults(which='list')

    list_table_parser.add_argument(
        '-m', '--method',     # 多一個 help 的選項：如果有的話，設定為 TRUE，之後會用到
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

    conversion_parser = subparsers.add_parser('convert')
    conversion_parser.set_defaults(which='convert')

    ## 之後，如果是要 conver 的話，後面還要有 VALUE FROM TO (+TO)
    ## 這些為 convert 修件下的 positional argument
    conversion_parser.add_argument(
        'value',
        metavar='VALUE',
        type=float,
        action='store',
        help='The value to convert'
    )

    conversion_parser.add_argument(
        'from_unit',
        metavar='FROM',
        action='store',
        type=str,
        help='Unit to convert from'
    )

    conversion_parser.add_argument(
        'to_units',
        metavar='TO',
        action='store',
        nargs='+',
        type=str,
        help='Unit(s) to convert to'
    )

    ## 所有輸入的參數入列到 propos 裏
    props = parser.parse_args()

    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)


def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print 'Unit table %s can convert between the units:'
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                '''
                還記得 lambda function 嗎？
                        self.to_base_unit['ev']        = lambda x: x / 6241506480000000000.0
                        self.from_base_unit['ev']      = lambda x: x * 6241506480000000000.0
                '''                           
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':')+1:conversion.index('\n')].strip()
            print '%s (%s)' % (unit, formula)
        else:
            print unit


def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print '%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit'])


In [45]:
conversion = inspect.getsource(get_table("time").from_base_unit['d'])
conversion

"        self.from_base_unit['d']   = lambda x: x / 86400.0\n"

In [46]:
formula = conversion[conversion.index(':')+1:conversion.index('\n')].strip()
formula

'x / 86400.0'

In [49]:
!unitconvert -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


## 我們先看幾個例子

In [50]:
!unitconvert energy list

Unit table %s can convert between the units:
btu
j
wh
hph
cal
ev
kcal


In [51]:
!unitconvert energy list -m

Unit table %s can convert between the units:
btu (x * 0.00094781707775)
j (base unit)
wh (x * 0.00027777777778)
hph (x * 3.7250614123e-7)
cal (x * 0.23900573614)
ev (x * 6241506480000000000.0)
kcal (x * 0.00023884589663)


In [52]:
! unitconvert energy convert 2500 kcal cal j ev

2500.000000 kcal = 2501673.040151 cal
2500.000000 kcal = 10466999.999890 j
2500.000000 kcal = 65329848325475080695971840.000000 ev


## 2.2 主幹parser 

### 之前在 Chapter 05 中，有寫過 CLI.py 是用 argv[-] 來處理輸入的參數，
### 還記得嗎？  for arg in sys.argv[1:]:

### 現在，用另外一種方式，先宣告一個parser用於儲存table的類別，之後所有的輸入都會進這個物件裏，而且，我們也可以準備一些類似 --help 等的資訊

In [74]:
from unitconverter.Converter import get_table, convert_units
import inspect ## 之後我們要將 lambda function 的指令讀出來用

parser = argparse.ArgumentParser(description='Tool for converting units')

parser.add_argument(
    'table',
    metavar='TABLE',
    action='store',
    type=str,
    help='Unit table to use in conversion'
)

_StoreAction(option_strings=[], dest='table', nargs=None, const=None, default=None, type=<type 'str'>, choices=None, help='Unit table to use in conversion', metavar='TABLE')

#### 另外定義list和convert兩種功能，依靠關鍵字"list"和"convert"以在兩者之間切換

In [75]:
subparsers = parser.add_subparsers(help='operation to be performed')

list_table_parser = subparsers.add_parser('list')
list_table_parser.set_defaults(which='list')

conversion_parser = subparsers.add_parser('convert')
conversion_parser.set_defaults(which='convert')

#### 變數props儲存用戶輸入的功能關鍵字，利用它來進行功能切換

In [76]:
!unitconvert -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [77]:
props = parser.parse_args()

if props.which == 'list':
    _run_unit_list(props)
elif props.which == 'convert':
    _run_conversion(props)

usage: ipykernel_launcher.py [-h] TABLE {list,convert} ...
ipykernel_launcher.py: error: too few arguments


SystemExit: 2

##### 查看幫助

In [78]:
!unitconvert -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [79]:
!unitconvert energy -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [80]:
!unitconvert TABLE -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [81]:
!unitconvert something -h

usage: unitconvert-script.py [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


### 2.3 list功能
#### 對於list功能，新增一個argument -m，讓程式可以輸出不同單位之間的轉換關係

In [82]:
list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

_StoreTrueAction(option_strings=['-m', '--method'], dest='method', nargs=0, const=True, default=False, type=None, choices=None, help='Also output the conversion method from the base unit', metavar=None)

##### 查看幫助

In [83]:
!unitconvert energy list -h

usage: unitconvert-script.py TABLE list [-h] [-m]

optional arguments:
  -h, --help    show this help message and exit
  -m, --method  Also output the conversion method from the base unit


##### 查看energy table中所有的單位

In [84]:
!unitconvert energy list

Unit table %s can convert between the units:
btu
j
wh
hph
cal
ev
kcal


##### 查看energy table中單位之間的轉換關係

In [85]:
!unitconvert energy list -m

Unit table %s can convert between the units:
btu (x * 0.00094781707775)
j (base unit)
wh (x * 0.00027777777778)
hph (x * 3.7250614123e-7)
cal (x * 0.23900573614)
ev (x * 6241506480000000000.0)
kcal (x * 0.00023884589663)


#### list功能本質上是依靠_run_unit_list方法來實現的
    *呼叫Converter.py中的get_table方法
    *當沒有argument -m傳入時，props.method為False，此時僅輸出各單位名稱
    *當有argument -m傳入時，props.method為True，此時輸出各個單位間的轉換關係

In [86]:
def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print 'Unit table %s can convert between the units:'
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip()
            print '%s (%s)' % (unit, formula)
        else:
            print unit

##### 仔細看看_run_unit_list方法的實現細節

In [87]:
table = get_table('energy')
unit = 'ev'
table.from_base_unit[unit]

<function unitconverter.unit_tables.energy.<lambda>>

In [88]:
conversion = inspect.getsource(table.from_base_unit[unit])
conversion

"        self.from_base_unit['ev']      = lambda x: x * 6241506480000000000.0\n"

In [89]:
conversion[conversion.index(':') + 1:conversion.index('\n')]

' x * 6241506480000000000.0'

In [90]:
conversion[conversion.index(':') + 1:conversion.index('\n')].strip()

'x * 6241506480000000000.0'

### 2.4 convert功能
#### 對於convert功能，新增arguments，用來儲存單位轉換中的相關數值

In [91]:
conversion_parser.add_argument(
    'value',
    metavar='VALUE',
    type=float,
    action='store',
    help='The value to convert'
)

conversion_parser.add_argument(
    'from_unit',
    metavar='FROM',
    action='store',
    type=str,
    help='Unit to convert from'
)

conversion_parser.add_argument(
    'to_units',
    metavar='TO',
    action='store',
    nargs='+',
    type=str,
    help='Unit(s) to convert to'
)

_StoreAction(option_strings=[], dest='to_units', nargs='+', const=None, default=None, type=<type 'str'>, choices=None, help='Unit(s) to convert to', metavar='TO')

##### 查看幫助

In [92]:
!unitconvert energy convert -h

usage: unitconvert-script.py TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


##### 類似convert_units('energy', 200, 'wh', ['j', 'hph'])的用法，使用CLI進行單位轉換
    注意順序

In [93]:
! unitconvert energy convert 2500 kcal cal j ev

2500.000000 kcal = 2501673.040151 cal
2500.000000 kcal = 10466999.999890 j
2500.000000 kcal = 65329848325475080695971840.000000 ev


#### convert功能本質上是依靠_run_conversion方法來實現的
    *呼叫Converter.py中的convert_units方法

In [94]:
def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print '%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit'])